In order to run this notebook with the correct PYTHONPATH, you can use
```
$ ./e-mission-jupyter.bash notebook
```

This notebook makes heavy use of the following python libraries.
- `pandas`: from the python computing stack (http://pandas.pydata.org/pandas-docs/stable/)
- `geojson`: standard JSON representation of geographic data (http://geojson.org/)
- `folium`: (https://github.com/python-visualization/folium) python bridge to leaflet (http://leafletjs.com/)

### Pick a user to work with ###

In [1]:
pwd

'/var/emission/e-mission-server'

In [2]:
import emission.core.get_database as edb
import pandas as pd
all_users = pd.DataFrame(list(edb.get_uuid_db().find({}, {"user_email":1, "uuid": 1, "_id": 0})))
all_users

storage not configured, falling back to sample, default configuration
Connecting to database URL localhost


,user_email,uuid
0,test_july_22,40267d4c-5fe9-423c-b18d-cde3d09c68c2
1,new_user_1,d0cbf849-f9b1-4ba1-83be-1a148ec56a1e
2,patgendre,91bc554a-4228-4afd-b2ca-be8eb7e33760
3,test_jul,40267d4c-5fe9-423c-b18d-cde3d09c68c2
4,null,eb3ef582-ae15-4b63-a29d-5ef690c1c42b
5,test_user_1,4aa5e987-ccce-4d6f-a845-aeb700a4b9db
6,,dea22f09-8a53-429e-89ed-f8de55aab29a
7,patgendre94@gmail.com,7a467c0f-b13a-4070-af58-44a921826c88
8,fabrice.hasiak@cerema.fr,a224fa8a-3a86-4140-ae50-beab3f5cf70a
9,transportsdufutur@gmail.com,ce51579f-42cf-4d97-a1ce-fa132c791ea9


In [3]:
test_user_id_2 = all_users.iloc[2].uuid
test_user_id_7 = all_users.iloc[7].uuid

In [4]:
from uuid import UUID

In [4]:
test_user_id = all_users.iloc[0].uuid # replace with UUID from above

If you want to work across multiple users, just do the same thing again

In [11]:
print("tre")

tre


### Preferred access technique

The preferred technique to access wrapper objects from the timeseries is to use the abstract timeseries interface. This makes it easier for us to switch to alternative timeseries implementations later. The timeseries is conceptually a set of streams, one for each of the types, primarily indexed by time. So you can query for all entries of a particular time within a specified time range.

In [5]:
import emission.storage.timeseries.abstract_timeseries as esta
import emission.storage.decorations.analysis_timeseries_queries as esda
import emission.core.wrapper.entry as ecwe
import emission.storage.decorations.trip_queries as esdt
import emission.storage.timeseries.timequery as estt

In [6]:
print(test_user_id_2)
ts_2 = esta.TimeSeries.get_time_series(test_user_id_2)
ts_7 = esta.TimeSeries.get_time_series(test_user_id_7)

91bc554a-4228-4afd-b2ca-be8eb7e33760


#### Accessing entries directly

In [7]:
# Get all cleaned trips for the first user
entry_it = ts_2.find_entries(["segmentation/raw_trip"], time_query=None)

All keys and their mapping to data model objects can be found in 
https://github.com/e-mission/e-mission-server/blob/master/emission/core/wrapper/entry.py

In [15]:
# recherche des tracés d'iti user 2 et 7 (patgendre)
entry_rawtrips = ts_2.find_entries(["analysis/cleaned_trip"], time_query=None)
for ct in entry_rawtrips:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    print ("jour-heure ",cte.data.start_local_dt, cte.data.duration)

=== Trip: {"coordinates": [5.3732571, 43.4798247], "type": "Point"} -> {"coordinates": [5.470528, 43.5180872], "type": "Point"}
jour-heure  LocalDate({'year': 2019, 'month': 1, 'day': 28, 'hour': 17, 'minute': 44, 'second': 31, 'weekday': 0, 'timezone': 'Europe/Brussels'}) 2347.4140045642853
=== Trip: {"coordinates": [5.470528, 43.5180872], "type": "Point"} -> {"coordinates": [5.4705162, 43.518066], "type": "Point"}
jour-heure  LocalDate({'year': 2019, 'month': 1, 'day': 28, 'hour': 18, 'minute': 26, 'second': 8, 'weekday': 0, 'timezone': 'Europe/Brussels'}) 1216.2246940135956
=== Trip: {"coordinates": [5.4657783, 43.5178936], "type": "Point"} -> {"coordinates": [5.4703689, 43.5181434], "type": "Point"}
jour-heure  LocalDate({'year': 2019, 'month': 1, 'day': 30, 'hour': 19, 'minute': 52, 'second': 40, 'weekday': 2, 'timezone': 'Europe/Brussels'}) 1305.3540000915527
=== Trip: {"coordinates": [5.4660641, 43.5177212], "type": "Point"} -> {"coordinates": [5.3736241, 43.4918444], "type": "P

In [19]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    print ("jour-heure ",cte.data.start_local_dt, cte.data.duration)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id_7, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

In [9]:
print (cte.data)

Cleanedtrip({'source': 'DwellSegmentationTimeFilter', 'end_ts': 1548920763.074, 'end_local_dt': {'year': 2019, 'month': 1, 'day': 31, 'hour': 8, 'minute': 46, 'second': 3, 'weekday': 3, 'timezone': 'Europe/Brussels'}, 'end_fmt_time': '2019-01-31T08:46:03.074000+01:00', 'end_loc': {'type': 'Point', 'coordinates': [5.3736241, 43.4918444]}, 'raw_trip': ObjectId('5c5bdb604e276d4ee60eff35'), 'start_ts': 1548919380.077, 'start_local_dt': {'year': 2019, 'month': 1, 'day': 31, 'hour': 8, 'minute': 23, 'second': 0, 'weekday': 3, 'timezone': 'Europe/Brussels'}, 'start_fmt_time': '2019-01-31T08:23:00.077000+01:00', 'start_loc': {'type': 'Point', 'coordinates': [5.4660641, 43.5177212]}, 'duration': 1382.9969999790192, 'distance': 10813.725022828585, 'start_place': ObjectId('5c5bdb624e276d4ee60f003e'), 'end_place': ObjectId('5c5bdb624e276d4ee60f003f')})


In [18]:
# Get all cleaned trips for the second user
entry_it = ts_1.find_entries(["analysis/cleaned_trip"], time_query=None)

In [19]:
for ct in entry_it:
    cte = ecwe.Entry(ct)
    print("=== Trip:", cte.data.start_loc, "->", cte.data.end_loc)
    section_it = esdt.get_sections_for_trip("analysis/cleaned_section", test_user_id, cte.get_id())
    for sec in section_it:
        print("  --- Section:", sec.data.start_loc, "->", sec.data.end_loc, " on ", sec.data.sensed_mode)

In [ ]:
# Get cleaned trips for the two users that started on 1st Aug UTC
import arrow

aug_1_tq = estt.TimeQuery("data.start_ts",
                          arrow.get("2017-08-01").timestamp, # start of range
                          arrow.get("2017-08-02").timestamp)  # end of range
entry_it = ts.find_entries(["analysis/cleaned_trip"], time_query=aug_1_tq)
entry_it_2 = ts_2.find_entries(["analysis/cleaned_trip"], time_query=aug_1_tq)
print("From %s -> %s, user %s had %d trips and user %s had %d trips" %
     (aug_1_tq.startTs, aug_1_tq.endTs, test_user_id, len(list(entry_it)), test_user_id_2, len(list(entry_it_2))))

#### Accessing a dataframe

In [22]:
# Get all cleaned trips for the first user
ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=None)
sm_df_2 = ts_2.get_data_df("analysis/smoothing", time_query=None)

In [12]:
len(ct_df_2)

4

In [23]:
ct_df_2.columns

Index(['_id', 'distance', 'duration', 'end_fmt_time', 'end_loc',
       'end_local_dt_day', 'end_local_dt_hour', 'end_local_dt_minute',
       'end_local_dt_month', 'end_local_dt_second', 'end_local_dt_timezone',
       'end_local_dt_weekday', 'end_local_dt_year', 'end_place', 'end_ts',
       'metadata_write_ts', 'raw_trip', 'source', 'start_fmt_time',
       'start_loc', 'start_local_dt_day', 'start_local_dt_hour',
       'start_local_dt_minute', 'start_local_dt_month',
       'start_local_dt_second', 'start_local_dt_timezone',
       'start_local_dt_weekday', 'start_local_dt_year', 'start_place',
       'start_ts', 'user_id'],
      dtype='object')

In [24]:
ct_df_2[["start_loc", "end_loc", "start_fmt_time", "end_fmt_time"]]

,start_loc,end_loc,start_fmt_time,end_fmt_time
0,"{'type': 'Point', 'coordinates': [5.3732571, 4...","{'type': 'Point', 'coordinates': [5.470528, 43...",2019-01-28T17:44:31.585995+01:00,2019-01-28T18:23:39+01:00
1,"{'type': 'Point', 'coordinates': [5.470528, 43...","{'type': 'Point', 'coordinates': [5.4705162, 4...",2019-01-28T18:26:08.775306+01:00,2019-01-28T18:46:25+01:00
2,"{'type': 'Point', 'coordinates': [5.4657783, 4...","{'type': 'Point', 'coordinates': [5.4703689, 4...",2019-01-30T19:52:40.300000+01:00,2019-01-30T20:14:25.654000+01:00
3,"{'type': 'Point', 'coordinates': [5.4660641, 4...","{'type': 'Point', 'coordinates': [5.3736241, 4...",2019-01-31T08:23:00.077000+01:00,2019-01-31T08:46:03.074000+01:00


In [34]:
# Get all cleaned trips for the second user
ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=None)
ct_df_2[["start_loc", "end_loc", "start_ts", "end_ts"]]

,start_loc,end_loc,start_ts,end_ts
0,"{'type': 'Point', 'coordinates': [5.3732571, 4...","{'type': 'Point', 'coordinates': [5.470528, 43...",1.548694e+09,1.548696e+09
1,"{'type': 'Point', 'coordinates': [5.470528, 43...","{'type': 'Point', 'coordinates': [5.4705162, 4...",1.548696e+09,1.548698e+09
2,"{'type': 'Point', 'coordinates': [5.4657783, 4...","{'type': 'Point', 'coordinates': [5.4703689, 4...",1.548874e+09,1.548876e+09
3,"{'type': 'Point', 'coordinates': [5.4660641, 4...","{'type': 'Point', 'coordinates': [5.3736241, 4...",1.548919e+09,1.548921e+09


In [ ]:
# Get cleaned trips for the two users that started on 1st Aug UTC
import arrow

aug_1_tq = estt.TimeQuery("data.start_ts",
                          arrow.get("2017-08-01").timestamp, # start of range
                          arrow.get("2017-08-02").timestamp)  # end of range
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=aug_1_tq)
ct_df_2 = ts_2.get_data_df("analysis/cleaned_trip", time_query=aug_1_tq)
print("From %s -> %s, user %s had %d trips and user %s had %d trips" %
     (aug_1_tq.startTs, aug_1_tq.endTs, test_user_id, len(ct_df), test_user_id_2, len(ct_df_2)))

In [ ]:
cs_df = ts.get_data_df("analysis/cleaned_section", time_query=None)

In [ ]:
len(cs_df)

In [ ]:
cs_df.columns

In [ ]:
cs_df[["start_loc", "end_loc", "start_ts", "end_ts", "sensed_mode"]]

### Direct mongodb queries

You can also use direct mongodb queries during exploratory work. I do ask that you create a storage decorator (`emission/storage/decorations`) when you submit a pull request for ongoing use

In [21]:
import emission.core.get_database as edb

In [22]:
edb.get_timeseries_db().find_one()

{'_id': ObjectId('5c49e7944e276d05c81a293b'),
 'user_id': '',
 'metadata': {'key': 'stats/server_api_time',
  'platform': 'server',
  'write_ts': 1548347284.757146,
  'time_zone': 'America/Los_Angeles',
  'write_local_dt': {'year': 2019,
   'month': 1,
   'day': 24,
   'hour': 8,
   'minute': 28,
   'second': 4,
   'weekday': 3,
   'timezone': 'America/Los_Angeles'},
  'write_fmt_time': '2019-01-24T08:28:04.757146-08:00'},
 'data': {'name': 'GET_/',
  'ts': 1548347284.75363,
  'reading': 0.02602386474609375}}

In [23]:
edb.get_timeseries_db().distinct("metadata.key")

['stats/server_api_time',
 'background/location',
 'background/filtered_location',
 'statemachine/transition',
 'background/motion_activity',
 'stats/pipeline_time',
 'stats/client_nav_event',
 'background/battery',
 'stats/client_time',
 'config/consent',
 'stats/client_error',
 'stats/server_api_error']

Note that in this case, you need to know whether to use the `timeseries` or the `analysis_timeseries` collection

In [24]:
edb.get_analysis_timeseries_db().distinct("metadata.key")

['segmentation/raw_trip',
 'segmentation/raw_place',
 'segmentation/raw_section',
 'segmentation/raw_stop',
 'analysis/smoothing',
 'analysis/recreated_location',
 'analysis/cleaned_section',
 'analysis/cleaned_stop',
 'analysis/cleaned_place',
 'analysis/cleaned_trip',
 'segmentation/raw_untracked',
 'analysis/cleaned_untracked',
 'inference/prediction',
 'analysis/inferred_section']

In [26]:
edb.get_analysis_timeseries_db().find({"user_id": test_user_id_2, "metadata.key": "analysis/cleaned_trip"}).count()

4

In particular, you can use this to access entries that are not in the timeseries

In [27]:
edb.get_uuid_db().distinct("uuid")

[UUID('40267d4c-5fe9-423c-b18d-cde3d09c68c2'),
 UUID('d0cbf849-f9b1-4ba1-83be-1a148ec56a1e'),
 UUID('91bc554a-4228-4afd-b2ca-be8eb7e33760')]

### Timeline

The trips and places maintain links to each other - e.g. `start_place`, `end_place`

In [ ]:
ct_df[["start_place", "end_place"]]

These are _primary key links_ to other entries in the database. It would be useful to have a doubly linked list representing this properly. The Timeline helps with that.

In [ ]:
import emission.storage.decorations.timeline as esdl

In [ ]:
trip_start_end_fuzz = 10 # seconds
ct_df = ts.get_data_df("analysis/cleaned_trip", time_query=None)
tl = esdl.get_cleaned_timeline(test_user_id, ct_df.iloc[0].start_ts - trip_start_end_fuzz, ct_df.iloc[-1].end_ts + trip_start_end_fuzz)

In [ ]:
for e in tl:
    if 'enter_ts' in e.data:
        # Must be place-like
        print(e.metadata.key, e.data.enter_fmt_time, "->", e.data.exit_fmt_time)
    else:
        print(e.metadata.key, e.data.start_fmt_time, "->", e.data.end_fmt_time)

In [ ]:
# The timeline is an iterator, so after it is consumed, it is empty
for e in tl:
    if 'enter_ts' in e.data:
        # Must be place-like
        print(e.metadata.key, e.data.enter_fmt_time, "->", e.data.exit_fmt_time)
    else:
        print(e.metadata.key, e.data.start_fmt_time, "->", e.data.end_fmt_time)

In [ ]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.first_trip().get_id())

In [ ]:
for e in stl:
    print(e.metadata.key)

In [ ]:
stl = esdt.get_cleaned_timeline_for_trip(test_user_id, tl.last_trip().get_id())

In [ ]:
for e in stl:
    print(e.metadata.key)

## Getting trip and section details ##

Once we have trip and section objects, we can retrieve the sensed data associated with them by querying for data in various streams that falls within the time ranges associated with the trip/section. Here again, our architecture of storing the analysis results as a separate datastream makes it easy to retrieve data at various levels of processing.

### Plot a processed trip or set of trips ###

In [35]:
import emission.analysis.plotting.geojson.geojson_feature_converter as gfc
import emission.analysis.plotting.leaflet_osm.our_plotter as lo

analysis.debug.conf.json not configured, falling back to sample, default configuration


In [44]:
first_trip_for_user_2 = ct_df.iloc[2]
first_trip_start_ts_2 = first_trip_for_user.start_ts
first_trip_end_ts_2 = first_trip_for_user.end_ts
trip_start_end_fuzz = 10 # seconds
trips_geojson_list = gfc.get_geojson_for_ts(test_user_id_2, first_trip_start_ts_2-trip_start_end_fuzz, ct_df_2.iloc[-1].end_ts+trip_start_end_fuzz)

In [45]:
len(trips_geojson_list)

4

In [46]:
map_list = lo.get_maps_for_geojson_trip_list(trips_geojson_list)

start_place 5c5bdb624e276d4ee60f0038
end_place 5c5bdb624e276d4ee60f003a
section 5c5bdb614e276d4ee60eff51
start_place 5c5bdb624e276d4ee60f003a
end_place 5c5bdb624e276d4ee60f003b
stop 5c5bdb624e276d4ee60effd3
stop 5c5bdb624e276d4ee60effd4
section 5c5bdb624e276d4ee60effa3
section 5c5bdb624e276d4ee60effb7
section 5c5bdb624e276d4ee60effc9
start_place 5c5bdb624e276d4ee60f003c
end_place 5c5bdb624e276d4ee60f003d
stop 5c5bdb624e276d4ee60f0004
section 5c5bdb624e276d4ee60effd7
section 5c5bdb624e276d4ee60efff0
start_place 5c5bdb624e276d4ee60f003e
end_place 5c5bdb624e276d4ee60f003f
section 5c5bdb624e276d4ee60f0007


In [47]:
len(map_list)

4

In [52]:
map_list[2]

In [49]:
map_list[-1]

In [ ]:
import branca.element as bre

In [ ]:
nrows = 2
ncols = 3
fig = bre.Figure()
for i, m in enumerate(map_list[:6]):
    fig.add_subplot(nrows,ncols,i+1).add_child(m)
fig

In [ ]:
nrows = 2
ncols = 3
fig = bre.Figure()
for i, map in enumerate(map_list[-6:]):
    fig.add_subplot(nrows,ncols,i+1).add_child(map)
fig

## Can you do better? ##


### Get locations with no processing, basic filtering and resampling for the first trip ###

In [ ]:
all_locs = ts.get_data_df("background/location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

filtered_locs = ts.get_data_df("background/filtered_location",
                               time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

resampled_locs = ts.get_data_df("analysis/recreated_location",
                                 time_query = esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))
print("Locations go from all = %d -> filtered = %d -> resampled = %d" % (len(all_locs),
                                                                         len(filtered_locs),
                                                                         len(resampled_locs)))

In [ ]:
all_locs[["_id", "latitude", "longitude", "fmt_time"]]

### Get the raw motion activity, in case you want to do different segmentation ###

In [ ]:
all_activity = ts.get_data_df("background/motion_activity",
                               esda.get_time_query_for_trip_like(
                                   "analysis/cleaned_section", ct_df.iloc[0]._id))

In [ ]:
all_activity.columns

In [ ]:
import emission.core.wrapper.motionactivity as ecwm

print("Found %d walking entries, %d on_foot entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.WALKING.value]), 
                                    len(all_activity[all_activity.type == ecwm.MotionTypes.ON_FOOT.value])))
print("Found %d motorized entries" % (len(all_activity[all_activity.type == ecwm.MotionTypes.IN_VEHICLE.value])))

### Plot the location points ###

In [ ]:
map_list = lo.get_maps_for_geojson_unsectioned([gfc.get_feature_list_from_df(all_locs),
                                                gfc.get_feature_list_from_df(filtered_locs),
                                                gfc.get_feature_list_from_df(resampled_locs)])

In [ ]:
fig = bre.Figure()
for i, map in enumerate(map_list):
    fig.add_subplot(1,3,i+1).add_child(map)
fig